# The optimization of the parameters 

In [91]:
import tensorflow as tf # for implementing optimization
import numpy as np 
import matplotlib.pyplot as plt 
import sys 

sys.path.insert(0, '/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main')



In [92]:


import parameters as pr  
import monteCarloIntegration as mc 



In [93]:
import sympy as sym 
from sympy import symbols, diff, lambdify 

import re 
import pandas as pd 

**Testattu**
1. metropolisSamplingFunction ja monteCarloIntegrationFunction, joilla on parametrit,  voivat korvata vanhemmat vastaavat funktiot. 

In [94]:
# testattu 
def gradientDescentFunc(expr, paramVariables: list, initialParamValues: list,learning_rate: float = 0.001,  limitForLoop: int = 100): 
    '''
    Compute gradient descent from sympy expression, which has symbols. 
    
    Gradient descent update algorithm: alpha_i = alpha_(i-1)- learning_rate * gradient(expr)
    
    - expr: sympy expression 
    - paramVariables: list of sympy symbols used in the "expr". e.g. [sym.Symbol('a'), sym.Symbol('b')]
    - paramValues: current values of the parameters. Acts as initial values.
    - learning rate 
    
    
    
    Example: simple usage 
    
        # Define the sympy symbols to be used in the function
        x = symbols('x')
        y = symbols('y')
        #Define the function in terms of x and y
        f1 = (x-2) ** 2 + (y-2)**2+5      # sympy expression 


        paramVariables = [x, y]
        initialParamValues = [3.0, 3.0]

        gradientDescentFunc(f1, paramVariables = paramVariables, initialParamValues = initialParamValues )
    
    
    '''
    
    
    # error check: whether the length of paramVariables is the same as initialParamValues 
    if (len(paramVariables) != len(initialParamValues)): 
        print('Error: Length of the list of initial values must be the same as number of symbolic variables!!!')
        return 
    
    
    partialDerivativeList = [diff(expr, variable) for variable in paramVariables ]   # list of partial derivatives with respect to possible variables in the expr
    
    paramValuesList = initialParamValues.copy()  # list of parameter values, each representing its own parameter e.g. [2.0, 2.0] for [alpha, beta]
    
    
    # dictionary with format: 
    #     {
    #         sym.Symbol('a'):3, 
    #         sym.Symbol('b'):4, 

    #     }
    
    
    
    
    # list of gradients for determining when to cut the loop 
    # if parameters reached minimum, gradient should be zero
    gradientList = np.random.randint(low = 100, size = len(paramValuesList))  # initiating gradientList with random large numbers

    
    
    counter = 0 
    #Perform gradient descent
    while ( np.array(gradientList) < 0.001).all() == False :         # if all gradients are bigger or equal to 0.1, continue descending 
        counter += 1 
        
        # clean the list
        gradientList = []
        
        # update each parameter value with negative gradient descent 
        for index in range(len(paramValuesList)): 
            # for later substitution when evaluating derivatives  
            substitutionDict = {paramVariables[i]:paramValuesList[i] for i in range(len(paramVariables))}
                
                
            # gradient-descenting one parameter 
            paramValuesList[index] -= partialDerivativeList[index].evalf(subs=substitutionDict)*learning_rate
            
            gradientList.append(partialDerivativeList[index].evalf(subs=substitutionDict))  # appending the gradient to the list 
        

        
        # if user not wanting for too much loops, or not wanting to reach the local minimum,
        # or loop last too long, break the loop 
        if counter >= limitForLoop: 
            break 
    
    
    return paramValuesList

    

# Global values 

In [95]:
historyTable = {'Energy': [], 'Variance':[], 'Std error of mean': [], 'parameters': []}

def saveToHistoryTable(energyAndErrors: tuple or list, params: float or list or tuple): 
    '''
    Save integration results to the table historyTable (global value)
    
    - energyAndErrors: energy and their errors in format (energy, variance, std error of mean)
    - parameters: parameters.
        - it is float-type if there is only one parameter 
        - it is list or tuple-type if there are more than one parameter
    '''
    
    
    historyTable['Energy'].append(energyAndErrors[0])
    historyTable['Variance'].append(energyAndErrors[1])
    historyTable['Std error of mean'].append(energyAndErrors[2])
    historyTable['parameters'].append(params)

# Algorithm (Helium, 1 parameter)

In [96]:
# all relying on FortranForm 


def probabilityFunction(x: np.ndarray, params: list): 
    particleOnexyz = x[0] 
    particleTwoxyz = x[1]
    
    
    return (
        sym.exp(-2*params[0]*(np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2)))
    )






def localEnergyFunction(x: np.ndarray, params: list ): 
    particleOnexyz = x[0] 
    particleTwoxyz = x[1]
    
    
    return (  
        -params[0]**2 - 2/np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) - 2/np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2) + 1/np.sqrt(particleOnexyz[0]**2 - 2*particleOnexyz[0]*particleTwoxyz[0] + particleTwoxyz[0]**2 + particleOnexyz[1]**2 - 2*particleOnexyz[1]*particleTwoxyz[1] + particleTwoxyz[1]**2 + particleOnexyz[2]**2 - 2*particleOnexyz[2]*particleTwoxyz[2] + particleTwoxyz[2]**2) + params[0]*(1/np.sqrt(particleOnexyz[0]**2 + particleOnexyz[1]**2 + particleOnexyz[2]**2) + 1/np.sqrt(particleTwoxyz[0]**2 + particleTwoxyz[1]**2 + particleTwoxyz[2]**2))
    
    )








In [97]:

# MUST BE FLOAT!!!
alpha = 2.0   # initial guess of the parameter
paramsValuesList = [alpha]
paramsVariableList = [sym.Symbol('a')]


patience = 1 # how many times we accept the energy to jump to higher value than the previous energy 


In [98]:

loopMark = 0 # which loop we are going on 

while patience >= 0: 
    

    configSamples_Metropolis = mc.metropolisSamplingFunction(coordinateValueRange = (-1, 1), 
                                                         numberOfParticles = 2, 
                                                         numberOfConfig = 3000, 
                                                         probabilityFunction = probabilityFunction, 
                                                        numberOfIterations= 1000, 
                                                          params =  paramsValuesList
                                                         ) 

    

    # integration approximation using current parameter value 
    integrationResultAsValue = mc.monteCarloIntegrationFunction(configSamples_Metropolis, localEnergyFunction = localEnergyFunction,  params= paramsValuesList, needError = True)
    # save the results to the table 
    saveToHistoryTable(integrationResultAsValue, paramsValuesList)



    # integration approximation with parameter as variable for optimization 
    # return tuple, where first is the integration result with parameters as variables
    integrationResultWithParameter = mc.monteCarloIntegrationFunction(configSamples_Metropolis, localEnergyFunction = localEnergyFunction,  params= paramsVariableList)


    



    # optimization: gradient descent 
    
    # replace the current parameter values with optimized parameter values 
    paramsValuesList = gradientDescentFunc(
        expr = integrationResultWithParameter[0],  # integration result with parameter 
         paramVariables= paramsVariableList,        # list of symbolic variables representing parameters to be optimized 
        initialParamValues= paramsValuesList        # current parameter values as float 
    )   
    
    
     # transfrom to python float. Avoid issues with sympy.float and numpy.ufunc incompatibility 
    paramsValuesList = list(map(float, paramsValuesList))     
                  
    
    # when we have at least two energy values in the table, we can compare them 
    if loopMark >= 1: 
        # if current calculated energy is bigger than energy calculated in the previous loop, reduce patience by 1
        if historyTable['Energy'][-1] > historyTable['Energy'][-2]: 
            patience -= 1
    
    print(loopMark) 
    
    loopMark += 1 

paramsValuesList [2.0]
0
paramsValuesList [2.0000052673891466]
1
paramsValuesList [1.989841683878321]
2
paramsValuesList [1.989877989583803]
3


In [99]:
historyTable

{'Energy': [-2.7525106939324964,
  -2.7150608497232955,
  -2.7670800496142323,
  -2.7534667470518457],
 'Variance': [1.0158604794611872,
  0.9921219181198913,
  1.0386948868176162,
  2.30538618802505],
 'Std error of mean': [0.01840163470511236,
  0.018185359662100824,
  0.01860730042051252,
  0.027721148292865084],
 'parameters': [[2.0],
  [2.0000052673891466],
  [1.989841683878321],
  [1.989877989583803]]}

# Testaus 

## Regex

In [ ]:
string = '-1.0*a**2 + 3.98813746927895*a - 6.72856476573052'
re.findall(r'[-]?\d+[.]{1}\d+', string.replace(' ', ''))
